# Introduction to network measures with networkx
with Miguel and Andy

___

In this interactive notebook, we will introduce some basic concepts of network science and network analysis while also learning how to actually apply them using the __networkx__ software package in Python. __Networkx__ is a Python software package that allows for generation, manipulation, and analysis of complex network data. We hope this will be possible to follow even if you don't yet have much (if any) programming experience, but do feel free to let us know if anything isn't clear!

At the beginning of any Python code, we usually need to import some packages which contain functions we wish to use in our code. In addition to __networkx__, we will import the Python library __numpy__ which lets us work easily with numerical vectors and arrays, as well as the __matplotlib__ library (with the __pyplot__ package) which we use for plotting data.

The code below will import the package networkx so that we can use throughout this notebook, and also instructs Python to recognize it by the shorthand name "nx" from now on. Similarly, we will assign "np" and "plt" as shorthand when calling functions from numpy and matplotlib, respectively.

In [ ]:

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt


___
___

## Creating a network

Below, we have included sample code with a small example network provided. However, you should replace this with a small example network of your own. Design your own small graph (having maybe 7 or 8 nodes), label the nodes, and modify the codes below to create your graph in networkx. Try to make an interesting structure where nodes have different numbers of links!

### Create a "Graph" object

To create a new network in networkx, we choose a name for the network object (let's go with "G" for graph) and then assign an empty graph to that variable.

In [ ]:

G = nx.Graph()


### Add nodes

The network object now exists and has a name, but it is still empty; it contains no nodes or edges. We can add new nodes to the network using networkx commands __add_node__ and __add_nodes_from__. The names of these nodes can be of various types: they can be numbers, or __strings__ (...of text, indicated by single or double quote marks), or other data types. In the examples below, we'll make them strings for now.

We can nodes one at a time using __add_node__:

In [ ]:

G.add_node('A')
G.add_node('B')


If we want to add multiple nodes at once, we can also add nodes _from_ a set (an unordered bunch of items enclosed by parentheses) or list variable (an ordered bunch of items enclosed by square brackets) using __add_nodes_from__.

In [ ]:

G.add_nodes_from( ['C','D','E','F','G','H','I'] )


We can retrieve the nodes of our network using __nodes__ command like so (where we use Python's __print__ command to display the result):

In [ ]:

print(G.nodes())


We can also remove nodes using __remove_node__ or __remove_nodes_from__ in the same way. It can be handy to use Python's function __len__ to measure the _length_ of the list to count the number of nodes.

In [ ]:

print( "Before, there were this many nodes : ", len(G.nodes()) )

G.remove_nodes_from(['H','I'])

print(G.nodes())

print( "After, there were this many nodes : ", len(G.nodes()) )


### Add edges

Edges between these nodes can be added and removed using similar commands __add_edge__ , __add_edges_from__ , __remove_edge__ , and __remove_edges_from__ . The main difference is that edges must be expressed as pairs of the nodes that we have already added to the network, formatted within parentheses and separated by a comma. Note that the order is not important.


In [ ]:

G.add_edge('A','C')
G.add_edge('B','C')

G.add_edges_from( [ ('C','D') , ('D','E') , ('E','F') , ('E','G') , ('F','G')  ] ) 

print( "We have added this many edges : " , len(G.edges()) )

print( "And those edges are :" , G.edges() )
      

### Graph visualization

The networkx function __draw__ can be used to create simple network visualizations. While lacking the features of task-specific visualization software such as Gephi, which is highly recommended especially when dealing with larger networks, it is nonetheless sometimes convenient to call __draw__ for simple tasks, like checking that we have correctly created the small network we intended to make.


In [ ]:

nx.draw( G , pos=nx.spring_layout(G) , with_labels = True ) 


___
___

## Network analysis: centrality measures

We have created a Graph object, given it some nodes, and then added edges between them to describe a network. We will now introduce some of networkx's commands that allow us to analyze the roles of the network's nodes and links.

### Degree centrality

_Degree centrality_ (or more commonly just " _degree_ ") counts the number of links a node has. For the network you just created, count the degrees of each node by hand first...

In networkx, we compute this with __degree__ as demonstrated below. Note that there are a couple different ways to accomplish the same thing...


In [ ]:

print( nx.degree(G) )

print( G.degree() )

# Compute degree for some subset of nodes:

print( G.degree( ['A','D','G'] ) )


___

### Betweenness centrality

It takes a lot more than just counting edges to really describe a node's role within a network. One of the most widely-used centrality measures is _betweenness centrality_ , so called because it captures the extent to which a node lies between other nodes within the network.

The compute the betweenness centrality of a node $x$, we ...

* Consider every pair of nodes within the network

* Figure out what are the shortest pathS between these two nodes (there may be more than one!)

* Calculate the fraction of these shortest paths that passes through the node $x$

* Add all of these fractions together to define the total betweenness centrality

* __Normalize:__ Divide this betweenness by $\frac{(N-1)(N-2)}{2}$

Mathematically, we can express this as

$$ b(x) = \sum_{s \neq t \neq x} \frac{\sigma_{st}(x)}{\sigma_{st}} $$

where $\sigma_{st}$ is the number of distinct shortest paths between nodes $s$ and $t$, and $\sigma_{st}(x)$ is the number of those that pass through $x$.

Try to compute this by hand (for 2 or 3 nodes, at least!) by identifying and counting shortest paths within your network.

We can implement this in a code as follows, with the help of the networkx function __all_shortest_paths__ , as shown below:


In [ ]:
BC = {}
BCN = {}

N = len(G.nodes())

for x in G.nodes():
    b = 0 
    for s in G.nodes():  
        for t in G.nodes():
            if (s!=x) and (t!=x) and (s!=t):
                n = 0
                SP = list(nx.all_shortest_paths( G , s , t ))
                for sp in SP:
                    if x in sp:
                        n += 1
                b += n/len(SP)
    BC[x] = b/2
    BCN[x] = (b/2)/((N-1)*(N-2)/2)
    
print("Betweenness centralities (not normalized): ",BC)
print("Betweenness centralities (normalized): ",BCN)
                

Networkx does all this for us with the function __betweenness_centrality__ :

In [ ]:

print("Betweenness centralities (not normalized): ", nx.betweenness_centrality( G , normalized=False ) )

print("Betweenness centralities (normalized): ", nx.betweenness_centrality( G ) )


___

#### Edge betweenness centrality

We can also apply the same concept as a centrality measure for _links_ by counting the shortest paths that pass through a given link, rather than a given node.

Try to compute edge betweenness centrality manually for a few edges from your network.

In networkx, we can do this with the function __edge_betweenness_centrality__ :


In [ ]:
print("Edge betweenness centralities (not normalized): ", nx.edge_betweenness_centrality( G , normalized=False ) )

print("Edge betweenness centralities (normalized): ", nx.edge_betweenness_centrality( G ) )

___

### Closeness centrality

_Closeness centrality_ measures how close a node is, on average, to the other nodes of the network. As in betweenness centrality, computing closeness centrality requires identifying the shortest paths between nodes in a network.

To compute closeness centrality for a node $x$, we

* Compute how "far" the node $x$ is from each other node in the network by finding the shortest path and recording its length.

* Add these together for all other nodes in the network to compute the total "farness" of $x$ from the rest of the network's nodes

* Take the reciprocal of "farness" as the node's "closeness"

* __Normalize:__ Divide "farness" by the total number of other nodes $N-1$, that is, multiply closeness by $N-1$

Mathematically, we can express this as

$$ c(x) = \frac{1}{\sum_{y \neq x} d(x,y)} $$

where $d(x,y)$ is the length of the shortest path between $x$ and $y$.

We can program a simple code to do this ourselves with the help of the networkx function __shortest_path_length__, like this...


In [ ]:
CC = {}
CCN = {}

for x in G.nodes():
    farness = 0
    for y in G.nodes():
        if (x!=y):
            farness += nx.shortest_path_length(G, x, y)
    CC[x] = 1/farness
    CCN[x] = (N-1)/farness
    
print( "Closeness centralities (not normalized) :", CC )
print( "Closeness centralities (normalized) :", CCN )

Networkx will do this automatically for us with the function __closeness_centrality__ :

In [ ]:
print( "Closeness centralities (normalized) :", nx.closeness_centrality(G) )

___

___

## Degree distributions

Moving away from individual node measures to get a more macroscopic description of a network, we will now look at the _degree distribution_ , which describes the relative abundance of nodes of various degrees within a network. This is often the first step of exploratory network analysis. We will get started by introducing a method for generating random, artificial networks, so we will have a larger network to examine.


### Erdos-Renyi random graph model

In the Erdos-Renyi model, we begin with $n$ unconnected nodes. We then consider each pair of nodes in the network, and randomly decide whether or not to connect them. The probability of forming a link between each pair is $p$. An Erdos-Renyi model graph can be generated in Networkx using the function __erdos_renyi_graph__ .


In [ ]:

n = 50
p = .1

GER = nx.erdos_renyi_graph( n, p )

nx.draw( GER )


We will now visualize the degree distribution for the Erdos-Renyi network we have generated, by taking plotting a histogram of the degree values for all nodes in the network using matplotlib's function __hist__ .

In [ ]:

degreelist = dict(GER.degree()).values()

degrange = np.arange( min(degreelist) - 1 , max(degreelist) + 2 )
degbins = .5*( degrange[1:] + degrange[:-1] )

plt.hist( degreelist , degbins , density=True )
plt.xlabel( 'Degree' , fontsize = 16 )
plt.ylabel( 'Frequency' , fontsize = 16 )
plt.title( 'Erdos-Renyi graph, $n=$' + str(n) + " , $p=$" + str(p) , fontsize=14 )
plt.show()


### Questions:

* What kind of degree distribution does the Erdos-Renyi random graph model produce?
* How do changing the parameters $n$ and $p$ affect the properties of the network and/or its degree distribution?

___

## Barabasi-Albert "scale-free" network model

In many real-world networks, links are not formed at random as they are in the Erdos-Renyi model. Instead, some underlying process leads nodes with many connections to form new connections faster than less-connected nodes can: "the rich get richer." The Barabasi-Albert model models this by generating networks according to _preferential attachment_ , meaning that as new nodes are added to a network, they have a greater probability or "preference" of attaching themselves to nodes with higher degree. A total of $n$ nodes are added to the network one by one. As each node is added, it forms $m$ connections to the nodes already present in the network, so that the probability of connecting to an existing node $i$ is

$$ p_i = \frac{k_i}{\sum_j k_j} $$

The resulting networks are often called "scale-free". In networkx, we can generate such a network using the function __barabasi_albert_graph__ .



In [ ]:

n = 50
m = 1

GBA = nx.barabasi_albert_graph( n , m )

nx.draw( GBA )


In [ ]:

degreelist = dict(GBA.degree()).values()

degrange = np.arange( min(degreelist) - 1 , max(degreelist) + 2 )
degbins = .5*( degrange[1:] + degrange[:-1] )

P,__,__ = plt.hist( degreelist , degbins , density=True )
plt.xlabel( 'Degree' , fontsize = 16 )
plt.ylabel( 'Frequency' , fontsize = 16 )
plt.title( 'Barabasi-Albert graph, $n=$' + str(n) + " , $m=$ " + str(m) , fontsize=14 )
plt.show()



## Power-law degree distribution

When networks form based on prefential attachment as in the Barabasi-Albert model, this tends to result in a special kind of degree distribution known as a _power law_ , which mathematically is of the form

$$ P(k) = A k^{-\gamma} $$

where $k$ are degree values, and $A$ and $\gamma$ are positive constants. Network analyses of scale-free networks often involve finding the value of the exponent $\gamma$ which best describes the network at hand. Below, we demonstrate how we might go about doing this. Taking the log of both sides of the power-law definition above gives

$$ \log{P(k)} = {-\gamma} \log{k} + \log{A}  $$

so a "log-log" plot of a power-law distribution should show a line having $-\gamma$ as its slope. To approximate $\gamma$, we  can thus find a best-fit line for a plot of $\log{k}$ vs. $\log{P(k)}$, where $P(k)$ comes from the histogram results above.


In [ ]:

bincenters = .5*(degbins[1:] + degbins[:-1])

yvals = [p for ind,p in enumerate(P) if (p!=0)]
xvals = [bincenters[ind] for ind,p in enumerate(P) if (p!=0)]

R = np.polyfit( np.log(xvals) , np.log(yvals) , 1 )

plt.plot( np.log(xvals) , np.log(yvals) , 'ko' , label='Data' )
plt.plot( np.log(xvals) , R[1]*np.log(xvals) + R[0] ,'ro-' , label='Powerlaw fit, $\gamma=$' + str(round(-R[0],4)))
plt.xlabel( 'Log Degree' , fontsize = 16 )
plt.ylabel( 'Log Frequency' , fontsize = 16 )
plt.title( 'Degree distribution (Log-Log plot)' , fontsize = 16 )
plt.legend()
plt.show()

plt.plot( bincenters , P , 'ko-' , label='Data' )
plt.plot( bincenters , [ np.exp(R[1])*x**R[0] for x in bincenters ] , 'ro-' , label='Powerlaw fit, $\gamma=$' + str(round(-R[0],4)))
plt.xlabel( 'Degree' , fontsize = 16 )
plt.ylabel( 'Frequency' , fontsize = 16 )
plt.title( 'Degree distribution' , fontsize = 16 )
plt.legend()
plt.show()


### Questions:

* How does the network visualization of Barabasi-Albert scale-free networks visibly differ from the Erdos-Renyi networks we saw previously?

* How do changing the parameters $n$ and $m$ affect the properties of the network and/or its degree distribution? How does the value of the power-law exponent $\gamma$ correspond to the properties of the network?

* Which of these network models (Erdos-Renyi or Barabasi-Albert) do you think would be a better model for real-world social network formation, and why?

* Networks with power-law-type degree distributions, or other phenomena with sizes or magnitudes that are described by power-law distributions, are called "scale-free". What might this refer to?

___

## Global network properties


### Small-worldness

Macroscopic descriptions of a network also involve.
One property shared by many real-world networks is the _small-world_ property, which means that any node in the network can be reached from any other node by traversing a relatively small number of links ( think ["Six Degrees of Kevin Bacon"](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) ).

While the idea of small-worldness is intuitive, it has several consequences that we can capture quantitatively. A network's _diameter_ is the length of the __longest of all the shortest paths__ between pairs of nodes in the network. Simalarly, it is also common to examine the __average shortest path length__ between all pairs of nodes in the network. Small-world networks should have relatively small values for network diameter, and shorter values for average shortest path length.

Real-world networks having a small-world property tend to have a high average [clustering coefficient](https://en.wikipedia.org/wiki/Clustering_coefficient) , which means that nodes tend to cluster together in localized, highly-interconnected groups or cliques.


### Watts-Strogatz model

This idea was initially introduced by Watts and Strogatz, who created a model for generating small-worldess into networks, which roughly goes like:

* Begin with a _ring lattice_ of $n$ nodes, where nodes are arranged in a ring and each is connected to its $k$ nearest neighbors.
* Rewire some of the edges to one another with probability $p$

In the ring lattice, nodes are only connected to their nearest neighbors; to traverse from one part of the network to another can thus require traversing a long series of local links between neighbors (just like driving from one end of town to another block by block within a grid of roads). Random rewirings introduce "long-range" connections (like adding a direct highway as shortcut between two previously-separate parts of town, allows us to bypass all the local roads); using these long-range connections to hop between faraway sections of the networks, different parts of the network can thus be reached by traversing a smaller amount of links.

In networkx, we can use the Watts-Strogatz model to generate networks by calling the function __connected_watts_strogatz_graph__ . Below, we will call this function twice: first, with the re-wiring probability set to $0$ so that we obtain the original ring lattice with no rewiring, and second, with a chosen rewiring probability $p$ so that we can see how the rewiring algorithm alters the properties of this original network.

In [ ]:

n = 50
k = 4
p = .1

GWS0 = nx.connected_watts_strogatz_graph(n, k, 0)

nx.draw(GWS0)


In [ ]:

GWS = nx.connected_watts_strogatz_graph(n, k, p)

nx.draw(GWS)


Let's calculate the _network diameter_ , _mean shortest path length_ , and _average clustering_ of the original network and compare it with the re-wired network.


In [ ]:

print("Ring lattice BEFORE re-wiring : ")
print("\tDiameter: ", nx.diameter(GWS0) )
print("\tMean shortest path length: ", nx.average_shortest_path_length(GWS0) )
print("\tMean clustering: ", nx.average_clustering(GWS0) )
print("\n")

print("After Watts-Strogatz rewiring : ")
print("\tDiameter: ", nx.diameter(GWS) )
print("\tMean shortest path length: ", nx.average_shortest_path_length(GWS) )
print("\tMean clustering: ", nx.average_clustering(GWS) )
print("\n")



### Questions

* What changes and what stays more or less the same in the above network measures, and how can you account for this?

* How do the results change if you adjust the parameters $n$, $k$, and $p$?

* Do you predict that the network models we have seen previously (Erdos-Renyi random networks, Barabasi-Albert random networks) will possess this small-world property? Which (if any) do you predict will have a smaller diameter? (Feel free to investigate by experimenting with the cell below)


In [ ]:

n = 200

p = .07
GER2 = nx.erdos_renyi_graph( n, p )
print("Erdos-Renyi network ( ",len(GER2.edges())," edges )")
print("\tDiameter : ",nx.diameter(GER2))
print("\tAverage shortest path length: ",round(nx.average_shortest_path_length(GER2),5))
print("\tAverage clustering coefficient: ",round(nx.average_clustering(GER2),5))
print("\tDensity: ",round(nx.density(GER2),5))
print("\n")

m = 7
GBA2 = nx.barabasi_albert_graph( n , m )
print("Barabasi-Albert network ( ",len(GBA2.edges())," edges )")
print("\tDiameter : ",nx.diameter(GBA2))
print("\tAverage shortest path length: ",round(nx.average_shortest_path_length(GBA2),5))
print("\tAverage clustering coefficient: ",round(nx.average_clustering(GBA2),5))
print("\tDensity: ",round(nx.density(GBA2),5))
